# Gaussian Mixture Model

Implement the expectation maximization (EM) algorithm for estimating the parameters of a Gaussian Mixture Model (GMM). The GMM density is given by
$$p(\mathbf{x}|\mathbf{θ}) = \sum_{k=1}^{K} w_k \mathcal{N}(\mathbf{x}; \mathbf{\mu}_k,\Sigma_k)$$

where $x \in R^d , \mathcal{N}(\mathbf{x};\mathbf{\mu},\Sigma)$ is the multivariate Gaussian distribution with mean vector $\mathbf{\mu}$ and covariance matrix $Σ$. The parameter set $\mathbf{\theta} = [ w_1 , \dots, w_d , \mathbf{μ}_1 ,\dots, \mathbf{μ}_K, Σ_1 , \dots, Σ_K]$ . Your program must accept as inputs the observation matrix $X$ (of size $d \times N$), the vector dimension $d$ and the mixture size $K$ as inputs, and output the estimated parameter set $\mathbf{\hat{\theta}}$. Generate $X$ on your own and experiment by varying your choices of $\mathbf{\theta}$.

In [91]:
import numpy as np

In [110]:
# Generate dataset
def generate_dataset(d, N, K):
    '''
    means for generating data: (K, d) array with each d-dimensional sub-array as mean vector 
    for kth distribution. The kth mean vector contains d samples drawn from a uniform distribution
    between k and k+1 - this will be easy for observing outputs of Expectation Maximization.
    '''
    known_means = np.array([ np.random.uniform(high=k+1, low=k, size=d) for k in np.arange(K)])
    # covariance matrices for generating data
    # (K, d,d) array; each dxd matrix is positive semi-definite - valid covariance matrix for kth distribution
    A = np.array([np.random.normal(1, 1, (d,d)) for k in np.arange(K)])
    known_covs = np.array([np.matmul(P.T, P) for P in A])
    # number of samples in kth distribution; rand_k is array of these number of samples per each distribution
    while(True): 
        # this is to generate random k's and make sure that sum of k's is be N
        rand_k = np.random.uniform(high=1, low=0, size=K)
        rand_k = N*rand_k/np.sum(rand_k)
        rand_k = np.rint(rand_k).astype(np.int)
        if np.sum(rand_k) == N : break
    # generate N d-dimensional Random Gaussian vectors with means and covariances above
    # this is a list of samples from each Gaussian 
    mix = [np.random.multivariate_normal(known_means[i], known_covs[i], size=rand_k[i]) for i in np.arange(K)]
    # put these vectors in a Nxd array
    X = np.concatenate(mix, axis=0)
    return X, known_means, known_covs

In [4]:
# X, means, covs = generate_dataset(d=5,N=1000, K=3)
# X.shape

(1000, 5)

In [ ]:
from Kmeans import Kmeans # Kmeans from previous assignment

Class for EM estimator on GMM.

Reference: Bishop's Pattern Recognition and Machine Learning. 

In [ ]:
class GMM():
    def __init__(self,data, d, K):
        self.d = d
        self.X = data
        self.K = K
        self.N = len(X)
        #initalization using kmeans
        kmeans = Kmeans(X=data, K=K)
        kmeans.cluster(n_iter=1)
        self.means = kmeans.means
        # covariances are initalized by finding covariances of each cluster
        mixtures = kmeans.get_clusters()
        self.covars = np.array([np.cov(mixture, rowvar=False) for mixture in mixtures])
        # inialize mixing coefficients by fractions of number of data points in each cluster
        self.w = np.array([np.mean(kmeans.assign_k == k) for k in np.arange(K)])
        self.ln_p = self.log_likelihood()
    
    def log_likelihood(self):
        Px = np.sum([np.log(
                np.sum(
                    [self.w[k] * self.normal_pdf(x, self.means[k], self.covars[k])
                     for k in range(self.K)]))
                for x in self.X])
        return Px
    
    def normal_pdf(self, x, mu, sigma): # multivariate Gaussian pdf function
        n = len(x)
        px = 1/np.sqrt((2*np.pi)*np.linalg.det(sigma))
        px *= np.exp(-0.5*np.matmul(np.matmul(np.transpose(x-mu), np.linalg.inv(sigma)),(x-mu)))
        return px
    
    def gamma(self, n, k): # E step: γ(z_nk)
        r = self.w[k]*self.normal_pdf(self.X[n], self.means[k], self.covars[k])
        r /= np.sum([self.w[i]*self.normal_pdf(self.X[n], self.means[i], self.covars[i]) for i in range(self.K)])
        return r

    def maximization(self): # M step
        Nk = np.array([np.sum([self.gamma(n,k) for n in range(self.N)]) for k in range(self.K)])
        # Update means  
        means_new = np.array([
            (1/Nk[k]) * np.sum([self.gamma(n,k)*self.X[n] for n in range(self.N)], axis=0) for k in range(self.K)
        ])
        # Update covariance matrices
        covars_new = np.array([
            (1/Nk[k]) * np.sum([self.gamma(n,k)*np.tensordot((X[n]-means_new[k]), (X[n]-means_new[k]), axes=0) 
                    for n in range(self.N)], axis=0) 
        for k in range(self.K)])
        # Update mixing coefficients
        w_new = Nk/self.N 
#         self.new_ln_p = self.log_likelihood()
        return w_new, means_new, covars_new
    
    def EM(self, threshold):
        count = 0
        self.w, self.means, self.covars = self.maximization()
        self.new_ln_p = self.log_likelihood()
        count +=1
        print('Iteration Count: ', count)
        print('Log likelihood error: ', (self.new_ln_p - self.ln_p))
        print('Log likelihood :', self.ln_p, self.new_ln_p)
        while (self.new_ln_p - self.ln_p) > threshold:
            self.ln_p = self.new_ln_p
            self.w, self.means, self.covars = self.maximization()
            self.new_ln_p = self.log_likelihood()
            count +=1
            print('Iteration Count:', count)
            print('Log likelihood error: ', (self.new_ln_p - self.ln_p))
            print('Log likelihood :', self.new_ln_p)
    
    def optimal_params(self):
        return self.w, self.means, self.covars

In [107]:
d = 2
K = 1
X, means, covars = generate_dataset(d=d, N=100, K=K)
model = GMM(data=X, d=d, K=K)
model.EM(threshold=1e-6)
####################################################
print('*********************************************')
print('Actual Means:')
print(means)
print('Number of mixtures: ', K)
print('Actual Covariances:')
print(covars)
print('-------------------------------')
W, means_hat, covars_hat = model.optimal_params()
print('Estimated Means:') # note: may not be in the same order of the above actual means
print(means_hat)
print('Number of mixtures: ', K)
print('Estimated Covariances:')
print(covars_hat)
print('Mixing Parameters:')
print(W)

Done
Log likelihood error:  0.005033585350133762
Log likelihood : -288.00536545712254 -288.0003318717724
Iteration count:  1
Iteration Count: 2
Log likelihood error:  0.0
Log likelihood : -288.0003318717724
*********************************************
Actual Means:
[[0.50396255 0.35975807]]
Number of mixtures:  1
Actual Covariances:
[[[5.54110627 1.79678892]
  [1.79678892 2.01709567]]]
-------------------------------
Estimated Means:
[[0.7155803  0.50986425]]
Number of mixtures:  1
Estimated Covariances:
[[[4.48467224 1.12631913]
  [1.12631913 1.80706505]]]
Mixing Parameters:
[1.]


In [108]:
d = 2
K = 3
X, means, covars = generate_dataset(d=d, N=1000, K=K)
model = GMM(data=X, d=d, K=K)
model.EM(threshold=1e-5)
####################################################
print('*********************************************')
print('Actual Means:')
print(means)
print('Number of mixtures: ', K)
print('Actual Covariances:')
print(covars)
print('-------------------------------')
W, means_hat, covars_hat = model.optimal_params()
print('Estimated Means:') # note: may not be in the same order of the above actual means
print(means_hat)
print('Number of mixtures: ', K)
print('Estimated Covariances:')
print(covars_hat)
print('Mixing Parameters:')
print(W)

Done
Log likelihood error:  38.23892020144558
Log likelihood : -2744.4542561304547 -2706.215335929009
Iteration count:  1
Iteration Count: 2
Log likelihood error:  13.96492084729016
Log likelihood : -2692.250415081719
Iteration Count: 3
Log likelihood error:  7.06842506392104
Log likelihood : -2685.181990017798
Iteration Count: 4
Log likelihood error:  4.155628311753844
Log likelihood : -2681.026361706044
Iteration Count: 5
Log likelihood error:  2.6706923037113484
Log likelihood : -2678.3556694023328
Iteration Count: 6
Log likelihood error:  1.8215879800281982
Log likelihood : -2676.5340814223046
Iteration Count: 7
Log likelihood error:  1.2978733454438043
Log likelihood : -2675.2362080768607
Iteration Count: 8
Log likelihood error:  0.9578018634229011
Log likelihood : -2674.278406213438
Iteration Count: 9
Log likelihood error:  0.7287945529810713
Log likelihood : -2673.549611660457
Iteration Count: 10
Log likelihood error:  0.5702732039353577
Log likelihood : -2672.9793384565214
Iter

Iteration Count: 84
Log likelihood error:  8.00847672997179
Log likelihood : -2598.5344309213106
Iteration Count: 85
Log likelihood error:  9.094244599691592
Log likelihood : -2589.440186321619
Iteration Count: 86
Log likelihood error:  10.152690663089743
Log likelihood : -2579.287495658529
Iteration Count: 87
Log likelihood error:  11.7608310334831
Log likelihood : -2567.526664625046
Iteration Count: 88
Log likelihood error:  14.545657472118819
Log likelihood : -2552.9810071529273
Iteration Count: 89
Log likelihood error:  18.361372008926082
Log likelihood : -2534.619635144001
Iteration Count: 90
Log likelihood error:  22.053516016235335
Log likelihood : -2512.566119127766
Iteration Count: 91
Log likelihood error:  24.295802410978467
Log likelihood : -2488.2703167167874
Iteration Count: 92
Log likelihood error:  24.67577176134546
Log likelihood : -2463.594544955442
Iteration Count: 93
Log likelihood error:  24.262215675597872
Log likelihood : -2439.332329279844
Iteration Count: 94
Log

In [114]:
d = 5
K = 10
X, means, covars = generate_dataset(d=d, N=2000, K=K)
model = GMM(data=X, d=d, K=K)
model.EM(threshold=1e-4)
####################################################
print('*********************************************')
print('Actual Means:')
print(means)
print('Number of mixtures: ', K)
print('Actual Covariances:')
print(covars)
print('-------------------------------')
W, means_hat, covars_hat = model.optimal_params()
print('Estimated Means:') # note: may not be in the same order of the above actual means
print(means_hat)
print('Number of mixtures: ', K)
print('Estimated Covariances:')
print(covars_hat)
print('Mixing Parameters:')
print(W)

Done
Log likelihood error:  212.79301396342998
Log likelihood : -16707.234835159 -16494.44182119557
Iteration count:  1
Iteration Count: 2
Log likelihood error:  123.45495088104326
Log likelihood : -16370.986870314528
Iteration Count: 3
Log likelihood error:  125.34061248641046
Log likelihood : -16245.646257828117
Iteration Count: 4
Log likelihood error:  162.90216686210988
Log likelihood : -16082.744090966007
Iteration Count: 5
Log likelihood error:  241.54601548204482
Log likelihood : -15841.198075483962
Iteration Count: 6
Log likelihood error:  378.43996693519875
Log likelihood : -15462.758108548764
Iteration Count: 7
Log likelihood error:  480.20282143470286
Log likelihood : -14982.55528711406
Iteration Count: 8
Log likelihood error:  481.77281905843483
Log likelihood : -14500.782468055626
Iteration Count: 9
Log likelihood error:  312.22930095393167
Log likelihood : -14188.553167101694
Iteration Count: 10
Log likelihood error:  66.20490133164276
Log likelihood : -14122.348265770052

Iteration Count: 84
Log likelihood error:  33.03983651630915
Log likelihood : -13690.908318633545
Iteration Count: 85
Log likelihood error:  57.05547878766629
Log likelihood : -13633.852839845878
Iteration Count: 86
Log likelihood error:  78.32489738109325
Log likelihood : -13555.527942464785
Iteration Count: 87
Log likelihood error:  41.11193174584514
Log likelihood : -13514.41601071894
Iteration Count: 88
Log likelihood error:  15.122352676347873
Log likelihood : -13499.293658042592
Iteration Count: 89
Log likelihood error:  3.1216056989687786
Log likelihood : -13496.172052343623
Iteration Count: 90
Log likelihood error:  1.0814312514794437
Log likelihood : -13495.090621092144
Iteration Count: 91
Log likelihood error:  0.7628280594672106
Log likelihood : -13494.327793032677
Iteration Count: 92
Log likelihood error:  0.6980429465911584
Log likelihood : -13493.629750086086
Iteration Count: 93
Log likelihood error:  0.7056922845840745
Log likelihood : -13492.924057801501
Iteration Count

Iteration Count: 167
Log likelihood error:  0.4361329262428626
Log likelihood : -13435.433035321112
Iteration Count: 168
Log likelihood error:  0.5915132365917088
Log likelihood : -13434.84152208452
Iteration Count: 169
Log likelihood error:  1.1701785123623267
Log likelihood : -13433.671343572158
Iteration Count: 170
Log likelihood error:  0.8873895969009027
Log likelihood : -13432.783953975257
Iteration Count: 171
Log likelihood error:  1.316292019932007
Log likelihood : -13431.467661955325
Iteration Count: 172
Log likelihood error:  0.9743859615227848
Log likelihood : -13430.493275993802
Iteration Count: 173
Log likelihood error:  0.2307567538755393
Log likelihood : -13430.262519239926
Iteration Count: 174
Log likelihood error:  0.12137306638578593
Log likelihood : -13430.14114617354
Iteration Count: 175
Log likelihood error:  0.08777284118332318
Log likelihood : -13430.053373332357
Iteration Count: 176
Log likelihood error:  0.07712309459748212
Log likelihood : -13429.97625023776
I

KeyboardInterrupt: 